## Baseline Models for Education Prediction

As a point of comparison, we also evaluate standard regression models trained directly on household-level features, without using simulated labels or clustering.

These models include:
- **Decision Tree**
- **Linear Regression**
- **Random Forest Regressor**
- **XGBoost**

Each baseline model is trained to predict the real percentage of higher education attainment (`Vyssi_vzdelani_real`) using only the observed household features available at the district level.

This allows us to evaluate whether the simulation-based approach improves over direct supervised learning when fine-grained labels are limited or incomplete.


In [67]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
import pandas as pd
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# === Čtyři baseline modely (pokud už jsou definované výš, tenhle blok můžeš přeskočit) ===
baseline_models = OrderedDict([
    ("Linear Regression", LinearRegression()),
    ("Decision Tree", DecisionTreeRegressor(random_state=42)),
    ("Random Forest", RandomForestRegressor(
        n_estimators=500,
        random_state=42,
        n_jobs=-1,
    )),
    ("Gradient Boosting", GradientBoostingRegressor(
        random_state=42,
    )),
])

# === Load education data ===
df_okres_edu = pd.read_excel("./DATA/vzdelani_with_percentage_only.xlsx")
df_kraj_edu = pd.read_excel("./DATA/krajske_vzdelani_percentage_only.xlsx")
df_households = pd.read_excel("./DATA/domacnosti.xlsx")

# --- sjednocení názvu sloupce s názvem území na "region" ---
for df in [df_okres_edu, df_kraj_edu, df_households]:
    if "region" not in df.columns:
        if "Unnamed: 0" in df.columns:
            df.rename(columns={"Unnamed: 0": "region"}, inplace=True)

# === Normalize region names ===
def normalize_region(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.lower()
         .str.strip()
         .str.replace(" ", "", regex=False)
         .str.normalize("NFKD")
         .str.encode("ascii", errors="ignore")
         .str.decode("utf-8")
    )

for df in [df_okres_edu, df_kraj_edu, df_households]:
    df["region"] = normalize_region(df["region"])

# === Rename target column for clarity ===
if "% vyšší vzdělání" in df_okres_edu.columns:
    df_okres_edu = df_okres_edu.rename(columns={"% vyšší vzdělání": "Edu_Share"})
if "% vyšší vzdělání" in df_kraj_edu.columns:
    df_kraj_edu = df_kraj_edu.rename(columns={"% vyšší vzdělání": "Edu_Share"})

# === Join households to KRAJ and OKRES tables ===
df_kraj_merged_edu = pd.merge(
    df_kraj_edu,
    df_households,
    on="region",
    how="left",
)

df_okres_merged_edu = pd.merge(
    df_okres_edu,
    df_households,
    on="region",
    how="left",
)

# === Feature columns from household statistics ===
# Kandidáti podle datové karty – použijeme jen ty, které opravdu v datech existují
feature_candidates_edu = [
    "celkem",
    "1 rodinou",
    "2 a více rodinami",
    "2 a vice rodinami",
    "domácnosti jednotlivců",
    "domacnosti jednotlivcu",
    "vícečlenné domácnosti",
    "viceclenne domacnosti",
]

# vezmeme průnik s reálnými sloupci po merge
feature_cols_edu = [
    col for col in feature_candidates_edu
    if col in df_kraj_merged_edu.columns and col in df_okres_merged_edu.columns
]

print("Použité feature sloupce pro education baseline:", feature_cols_edu)

# Drop rows with missing features or target
df_kraj_merged_edu = df_kraj_merged_edu.dropna(subset=feature_cols_edu + ["Edu_Share"])
df_okres_merged_edu = df_okres_merged_edu.dropna(subset=feature_cols_edu + ["Edu_Share"])

# === Training data (KRAJ level) ===
X_kraj_edu = df_kraj_merged_edu[feature_cols_edu]
y_kraj_edu = df_kraj_merged_edu["Edu_Share"]

# === Evaluation data (OKRES level) ===
X_okres_edu = df_okres_merged_edu[feature_cols_edu]
y_okres_edu = df_okres_merged_edu["Edu_Share"]

# === Scale features ===
scaler_edu = StandardScaler()
X_kraj_edu_scaled = scaler_edu.fit_transform(X_kraj_edu)
X_okres_edu_scaled = scaler_edu.transform(X_okres_edu)

# === Evaluate 4 baseline models ===
print("=== Baseline models (Kraj → Okres education) ===")
mae_edu_baselines = OrderedDict()

for name, model in baseline_models.items():
    model.fit(X_kraj_edu_scaled, y_kraj_edu)
    y_pred_edu = model.predict(X_okres_edu_scaled)
    mae = mean_absolute_error(y_okres_edu, y_pred_edu)
    mae_edu_baselines[name] = mae
    print(f"{name} MAE: {mae:.4f}")

mae_edu_baselines


Použité feature sloupce pro education baseline: ['celkem', '1 rodinou']
=== Baseline models (Kraj → Okres education) ===
Linear Regression MAE: 4.8877
Decision Tree MAE: 4.4086
Random Forest MAE: 3.4579
Gradient Boosting MAE: 4.3861


OrderedDict([('Linear Regression', 4.8877336740723765),
             ('Decision Tree', 4.408625676240569),
             ('Random Forest', 3.4579310026126233),
             ('Gradient Boosting', 4.386129327871837)])

In [68]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Load data
krajske_vzdelani = pd.read_excel("./DATA/krajske_vzdelani_percentage_only.xlsx")
okresni_vzdelani = pd.read_excel("./DATA/vzdelani_with_percentage_only.xlsx")
domacnosti = pd.read_excel("./DATA/domacnosti.xlsx")

# Normalize function
def normalize_text(series):
    return (
        series.str.lower().str.replace(" ", "")
        .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
    )

# Normalize names
krajske_vzdelani.columns = ["Kraj", "Vyssi_vzdelani"]
krajske_vzdelani["Kraj"] = normalize_text(krajske_vzdelani["Kraj"])
okresni_vzdelani.columns = ["Okres", "Vyssi_vzdelani"]
okresni_vzdelani = okresni_vzdelani[~okresni_vzdelani["Okres"].str.contains("Česká republika", case=False)]
okresni_vzdelani["Okres"] = normalize_text(okresni_vzdelani["Okres"])
domacnosti = domacnosti.rename(columns={"region": "Okres"})
domacnosti["Okres"] = normalize_text(domacnosti["Okres"].astype(str))

# Remove aggregate rows
domacnosti_okres = domacnosti[~domacnosti["Okres"].str.contains("kraj|ceskarepublika")].copy()

# Okres to Kraj mapping
okres_to_kraj_map = {
    "hlavnimestopraha": "hlavnimestopraha",
    "benesov": "stredoceskykraj",
    "beroun": "stredoceskykraj",
    "kladno": "stredoceskykraj",
    "kolin": "stredoceskykraj",
    "kutnahora": "stredoceskykraj",
    "melnik": "stredoceskykraj",
    "mladaboleslav": "stredoceskykraj",
    "nymburk": "stredoceskykraj",
    "praha-vychod": "stredoceskykraj",
    "praha-zapad": "stredoceskykraj",
    "pribram": "stredoceskykraj",
    "rakovnik": "stredoceskykraj",
    "ceskebudejovice": "jihoceskykraj",
    "ceskykrumlov": "jihoceskykraj",
    "jindrichuvhradec": "jihoceskykraj",
    "pisek": "jihoceskykraj",
    "prachatice": "jihoceskykraj",
    "strakonice": "jihoceskykraj",
    "tabor": "jihoceskykraj",
    "domazlice": "plzenskykraj",
    "klatovy": "plzenskykraj",
    "plzen": "plzenskykraj",
    "plzen-jih": "plzenskykraj",
    "plzen-mesto": "plzenskykraj",
    "plzen-sever": "plzenskykraj",
    "rokycany": "plzenskykraj",
    "tachov": "plzenskykraj",
    "cheb": "karlovarskykraj",
    "karlovyvary": "karlovarskykraj",
    "sokolov": "karlovarskykraj",
    "decin": "usteckykraj",
    "chomutov": "usteckykraj",
    "litomerice": "usteckykraj",
    "louny": "usteckykraj",
    "most": "usteckykraj",
    "teplice": "usteckykraj",
    "usti": "usteckykraj",
    "ceskalipa": "libereckykraj",
    "jablonecnadnisou": "libereckykraj",
    "liberec": "libereckykraj",
    "semily": "libereckykraj",
    "hradeckralove": "kralovehradeckykraj",
    "jicin": "kralovehradeckykraj",
    "nachod": "kralovehradeckykraj",
    "rychnovnadkneznou": "kralovehradeckykraj",
    "trutnov": "kralovehradeckykraj",
    "chrudim": "pardubickykraj",
    "pardubice": "pardubickykraj",
    "svitavy": "pardubickykraj",
    "ustranadlabem": "pardubickykraj",
    "havlickuvbrod": "krajvysocina",
    "jihlava": "krajvysocina",
    "pelhrimov": "krajvysocina",
    "trebic": "krajvysocina",
    "zdarnadsazavou": "krajvysocina",
    "blansko": "jihomoravskykraj",
    "brno": "jihomoravskykraj",
    "brnomesto": "jihomoravskykraj",
    "brno-venkov": "jihomoravskykraj",
    "breclav": "jihomoravskykraj",
    "hodonin": "jihomoravskykraj",
    "vyskov": "jihomoravskykraj",
    "znojmo": "jihomoravskykraj",
    "jesenik": "olomouckykraj",
    "olomouc": "olomouckykraj",
    "prostejov": "olomouckykraj",
    "prerov": "olomouckykraj",
    "sumperk": "olomouckykraj",
    "bruntal": "moravskoslezskykraj",
    "frydekmistek": "moravskoslezskykraj",
    "karvina": "moravskoslezskykraj",
    "novyjicin": "moravskoslezskykraj",
    "opava": "moravskoslezskykraj",
    "ostrava": "moravskoslezskykraj",
    "ustinadlabem": "usteckykraj",
    "ustinadorlici": "pardubickykraj",
    "brno-mesto": "jihomoravskykraj",
    "kromeriz": "zlinskykraj",
    "uherskehradiste": "zlinskykraj",
    "vsetin": "zlinskykraj",
    "zlin": "zlinskykraj",
    "frydek-mistek": "moravskoslezskykraj",
    "ostrava-mesto": "moravskoslezskykraj",
}

domacnosti_okres["Kraj"] = domacnosti_okres["Okres"].map(okres_to_kraj_map)

# Prepare training data at Kraj level
df_train = pd.merge(domacnosti_okres, krajske_vzdelani, on="Kraj", how="inner")
X_kraj = df_train.drop(columns=["Okres", "Kraj", "Vyssi_vzdelani"])
y_kraj = df_train["Vyssi_vzdelani"]

# Scale features
scaler = StandardScaler()
X_kraj_scaled = scaler.fit_transform(X_kraj)

# Prepare Okres features
X_okres = domacnosti_okres.drop(columns=["Okres", "Kraj"])
X_okres_scaled = scaler.transform(X_okres)

# Initialize results dataframe with Okres names
results = domacnosti_okres[["Okres"]].copy()

# Merge with actual values
results = pd.merge(results, okresni_vzdelani, on="Okres", how="left")
results = results.rename(columns={"Vyssi_vzdelani": "Actual_Education"})

# Train and predict with each model
models = {
    "Decision_Tree": DecisionTreeRegressor(random_state=42),
    "Linear_Regression": LinearRegression(),
    "Random_Forest": RandomForestRegressor(random_state=42),
    "Gradient_Boosting": GradientBoostingRegressor(random_state=42)
}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_kraj_scaled, y_kraj)
    predictions = model.predict(X_okres_scaled)
    results[f"Predicted_{model_name}"] = predictions

# Calculate absolute errors for each model
for model_name in models.keys():
    results[f"AbsError_{model_name}"] = abs(
        results["Actual_Education"] - results[f"Predicted_{model_name}"]
    )

# Save to CSV
output_file = "okres_predictions_all_models.csv"
results.to_csv(output_file, index=False)
print(f"\n✅ Results saved to {output_file}")
print(f"Total okresy: {len(results)}")
print(f"\nColumns: {list(results.columns)}")

# Display summary statistics
print("\n" + "="*60)
print("SUMMARY - Mean Absolute Error by Model:")
print("="*60)
for model_name in models.keys():
    valid_results = results.dropna(subset=["Actual_Education", f"Predicted_{model_name}"])
    mae = valid_results[f"AbsError_{model_name}"].mean()
    print(f"{model_name:20s}: {mae:.4f}")

print("\n" + "="*60)
print("Preview of results:")
print("="*60)
print(results.head(10).to_string())

Training Decision_Tree...
Training Linear_Regression...
Training Random_Forest...
Training Gradient_Boosting...

✅ Results saved to okres_predictions_all_models.csv
Total okresy: 77

Columns: ['Okres', 'Actual_Education', 'Predicted_Decision_Tree', 'Predicted_Linear_Regression', 'Predicted_Random_Forest', 'Predicted_Gradient_Boosting', 'AbsError_Decision_Tree', 'AbsError_Linear_Regression', 'AbsError_Random_Forest', 'AbsError_Gradient_Boosting']

SUMMARY - Mean Absolute Error by Model:
Decision_Tree       : 2.9372
Linear_Regression   : 2.6812
Random_Forest       : 2.9146
Gradient_Boosting   : 2.8852

Preview of results:
              Okres  Actual_Education  Predicted_Decision_Tree  Predicted_Linear_Regression  Predicted_Random_Forest  Predicted_Gradient_Boosting  AbsError_Decision_Tree  AbsError_Linear_Regression  AbsError_Random_Forest  AbsError_Gradient_Boosting
0  hlavnimestopraha         33.695987                33.695987                    33.458994                29.654746      

In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.metrics import r2_score

# Create directories if they don't exist
os.makedirs('plotsBaseline', exist_ok=True)

# Read the CSV file
df = pd.read_csv('okres_predictions_all_models.csv')

def safe_r2_score(y_true, y_pred):
    try:
        mask = ~(np.isnan(y_true) | np.isnan(y_pred) | np.isinf(y_true) | np.isinf(y_pred))
        return r2_score(y_true[mask], y_pred[mask])
    except Exception as e:
        print(f"R² calculation error: {e}")
        return np.nan

def plot_prediction_comparison(df, actual_column, predicted_columns,
                                figsize=(20, 10),
                                dpi=300):
    plt.figure(figsize=figsize, dpi=dpi)

    # Create subplots for each model
    for i, (model, predicted_column) in enumerate(predicted_columns.items(), 1):
        plt.subplot(2, 2, i)

        # Remove any potential NaN values
        valid_data = df.dropna(subset=[actual_column, predicted_column])

        # Scatter plot
        plt.scatter(valid_data[actual_column], valid_data[predicted_column],
                    alpha=0.7, s=30)  # Reduced marker size

        # Diagonal line
        min_val = min(valid_data[actual_column].min(), valid_data[predicted_column].min())
        max_val = max(valid_data[actual_column].max(), valid_data[predicted_column].max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)

        plt.title(f'{model} Baseline Prediction vs Actual', fontsize=10)
        plt.xlabel('Actual Values', fontsize=8)
        plt.ylabel('Predicted Values', fontsize=8)

        # Calculate metrics
        mae = np.mean(np.abs(valid_data[actual_column] - valid_data[predicted_column]))
        r2 = safe_r2_score(valid_data[actual_column], valid_data[predicted_column])

        plt.annotate(f'MAE: {mae:.2f}\nR²: {r2:.2f}',
                     xy=(0.05, 0.95),
                     xycoords='axes fraction',
                     fontsize=8)

        # Improve tick label readability
        plt.tick_params(axis='both', which='major', labelsize=6)

        # Add grid for better readability
        plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)

    plt.tight_layout()
    plt.savefig(os.path.join('plotsBaseline', 'education_baseline_prediction_comparison.png'),
                dpi=dpi, bbox_inches='tight')
    plt.close()

def plot_error_distribution(df, actual_column, predicted_columns,
                             figsize=(20, 10),
                             dpi=300):
    plt.figure(figsize=figsize, dpi=dpi)

    # Calculate and plot absolute errors
    for i, (model, predicted_column) in enumerate(predicted_columns.items(), 1):
        plt.subplot(2, 2, i)

        # Calculate absolute errors
        abs_errors = np.abs(df[actual_column] - df[predicted_column])

        # Plot error distribution
        sns.histplot(abs_errors, kde=True)

        # Calculate statistical metrics
        mean_error = np.mean(abs_errors)
        median_error = np.median(abs_errors)

        plt.title(f'{model} Absolute Error Distribution', fontsize=10)
        plt.xlabel('Absolute Error', fontsize=8)
        plt.ylabel('Frequency', fontsize=8)

        # Annotate with statistical information
        plt.annotate(f'Mean Error: {mean_error:.2f}\nMedian Error: {median_error:.2f}',
                     xy=(0.05, 0.95),
                     xycoords='axes fraction',
                     fontsize=8,
                     verticalalignment='top')

        # Improve tick label readability
        plt.tick_params(axis='both', which='major', labelsize=6)

    plt.tight_layout()
    plt.savefig(os.path.join('plotsBaseline', 'education_baseline_error_distribution.png'),
                dpi=dpi, bbox_inches='tight')
    plt.close()

def print_model_performance(df, actual_column, predicted_columns):
    print("Baseline Model Performance Summary:")
    for model, predicted_column in predicted_columns.items():
        abs_error = np.abs(df[actual_column] - df[predicted_column])

        print(f"\n{model} Baseline:")
        print(f"  Mean Absolute Error: {abs_error.mean():.4f}")
        print(f"  Median Absolute Error: {np.median(abs_error):.4f}")
        print(f"  Max Absolute Error: {abs_error.max():.4f}")

        # Calculate R² with error handling
        r2 = safe_r2_score(df[actual_column], df[predicted_column])
        print(f"  R² Score: {r2:.4f}")

# Define predicted columns
predicted_columns = {
    'Decision Tree': 'Predicted_Decision_Tree',
    'Linear Regression': 'Predicted_Linear_Regression',
    'Random Forest': 'Predicted_Random_Forest',
    'Gradient Boosting': 'Predicted_Gradient_Boosting'
}

# Actual column name
actual_column = 'Actual_Education'

# Generate plots and analysis
plot_prediction_comparison(df, actual_column, predicted_columns)
plot_error_distribution(df, actual_column, predicted_columns)
print_model_performance(df, actual_column, predicted_columns)

Baseline Model Performance Summary:

Decision Tree Baseline:
  Mean Absolute Error: 2.9372
  Median Absolute Error: 2.3800
  Max Absolute Error: 11.1941
  R² Score: 0.3191

Linear Regression Baseline:
  Mean Absolute Error: 2.6812
  Median Absolute Error: 2.2568
  Max Absolute Error: 11.3685
  R² Score: 0.4333

Random Forest Baseline:
  Mean Absolute Error: 2.9146
  Median Absolute Error: 2.3094
  Max Absolute Error: 12.9529
  R² Score: 0.3354

Gradient Boosting Baseline:
  Mean Absolute Error: 2.8852
  Median Absolute Error: 2.2540
  Max Absolute Error: 11.2702
  R² Score: 0.3392
